전처리 - 불용어, 불필요 삭제<br />

임베딩  <br />
BOW - 문자 카운팅 <br />
tf-idf -조건부확률 , 정밀도 증가<br />
n-gram <br />

### 네이버 영화 리뷰를 활용한 감정 분석

1) koNLP    2)tf-idf를 활용한 임베딩<br />

-예제 목표 : 가능하면 해당 데이터를 기반으로 간단한 웹 페이지도 만들어보자<br />

- koNLP 주요 라이브러리 설치<br />
pip install konlpy (한국어의 경우 조사랑 단어를 분리하기 위한 도구)<br />
pip install joblib (기울기를 고정적인 값으로 ,확률공간도 고정적인 값이다./직렬화)<br />

In [16]:
import numpy as np         #숫자 처리
import pandas as pd        #데이터 처리
import re                  # 문자열 처리-정규식

from konlpy.tag import Okt            #트위터에서 만든 한국어 처리기

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score


### 데이터 불러오기-table형식으로 데이터 받기

In [17]:
train_df= pd.read_table("data/ratings_train.txt")
test_df= pd.read_table("data/ratings_test.txt")
train_df.head(10)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


### 전처리-숫자데이터 없애고 문자열만 남기기(apply, lambda)

In [18]:
train_df= train_df.fillna(" ")
test_df= test_df.fillna(" ")

In [19]:
# 정규표현식 
# 프론트분야에서는 이메일, 전화번호에서 숫자랑 -,.를 분리할 수 있어야 한다.
re.sub(r"\d+", " ","123 456 하하하")

'    하하하'

In [20]:
train_df["document"]

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 150000, dtype: object

In [21]:
# 교재에 숫자는 의미가 없다고 지워라고 했기에 코드가 아래와 같다.
train_df["document"]=train_df["document"].apply(lambda x: re.sub(r"\d+", " ",x))
test_df["document"]=test_df["document"].apply(lambda x: re.sub(r"\d+", " ",x))

### 임베딩

In [22]:

okt= Okt()

def tw_tokenzier(text):
    tokenzier_ko = okt.morphs(text)
    return tokenzier_ko

In [23]:
#머신러닝에서는 max_df =0.9 /딥러닝에서는  max_df =0.3

# n-gram과 tf-idf(bow를 바탕으로 하기에 단어별로 띄운다.)
#유니그램(단일 단어)과 빅그램(연속 단어 쌍)을 모두 추출/3글자미만은 무시/문서의 90%보다 많은 비율로 나타나는 용어를 무시
tfidf_vect= TfidfVectorizer(tokenizer=tw_tokenzier,ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df["document"])


tfidf_matrix_train=tfidf_vect.transform(train_df["document"])

c:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


실무에서 쓰기에는 okt를 활용한 임베딩과정은 시간이 너무 많이 걸린다.

### 학습

In [24]:
lf = LogisticRegression(C=3.5, random_state=42)
lf.fit(tfidf_matrix_train, train_df["label"])


LogisticRegression(C=3.5, random_state=42)

In [25]:
p_text= tfidf_vect.transform(["이 영화 재미있어요"])

In [26]:
lf.predict(p_text)

array([1], dtype=int64)

In [27]:
p_text= tfidf_vect.transform(["이 영화 별로인듯"])

In [28]:
lf.predict(p_text)

array([0], dtype=int64)

In [29]:



tfidf_matrix_test=tfidf_vect.transform(test_df["document"])

In [30]:
preds = lf.predict(tfidf_matrix_test)
accuracy_score(test_df["label"], preds)

0.86182

### 01모델 만들기
["이 영화 별로인듯"] 1개만 돌리면 실행이 빠른데 test_df["document"] 다 돌리니 시간이 꽤나 걸린다. <br />
지금까지는 데이터전처리, 머신러닝학습, 결과확인이었다. <br />
학습시간을 줄이기 위해 joblib을 이용한다.기울기를 고정적인 값으로 ,확률공간도 고정적인 값이다.<br />

직렬화 : 어떤 것을 이진파일로 변환, 단 처음 지정한 언어로만 읽을 수 있다. <br/>
자바에서 직렬화(Serialization)는 객체의 상태를 바이트 스트림으로 변환하는 과정을 말합니다

In [33]:
#model폴더 만들고 실행
#아래 코드의 직렬화는 파이썬으로만 읽을 수 있는 직렬화이다.
import joblib
joblib.dump(lf, "model/lf.pkl")
joblib.dump(tfidf_vect,"model/tfidf_vect.pkl")


['model/tfidf_vect.pkl']

### 02 서비스 설계

input : 특성변수(많으면 학습률은 높지만, 특성변수 개수만큼 사용자로부터 입력받아야 하기에  사용자가 입력해야하는 값의 개수가 많아진다.)
데이터 전처리
머신러닝 학습
output: 확률 


### 03 구현-웹 기반
사용자->UI(특성변수)->HTTPS->Spring서비스<-RestController->추론서비스 <br/>
                                  Spring서비스->API->추론서비스   <br/>
                                    Spring서비스<-REST<-추론서비스 

### 03 추론기 구현
*템플릿 엔진: SpringBoot의 템플릿 엔진은 thymeleaf /jsp에서 {}안에 변수 넣어 서버 구동 시 값을 받는 형태 <br/>
*REST/API

### 03 구조<br/>
 model/        pickle<br/>
 statics/      대부분 모든 파일<br/>
 templates/    html<br/>
 app.py/<br/>